In [1]:
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
from pylab import *
import json
import h5py
import imageio
from IPython.display import Image
import os

import glob
import matplotlib.pyplot as plt
#from tifffile import imsave
import h5py

import sklearn
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K 
from tensorflow.keras.utils import to_categorical

In [3]:
HeadMRI_DIR = "D:/DataSet/HeadSeg/Raw_HeadSet/MRI/"
Headlabel_DIR = "D:/DataSet/HeadSeg/Raw_HeadSet/new_Label/"
AUGMENT_DIR = "D:/DataSet/HeadSeg/Raw_HeadSet/Augmentation/"

TRAIN_DATA_DIR = "/Volumes/MX500_1TB/DataSet/3DBrainTissueSegmentation/train/"
VALID_DATA_DIR = "/Volumes/MX500_1TB/DataSet/3DBrainTissueSegmentation/valid/"
TEST_DATA_DIR = "/Volumes/MX500_1TB/DataSet/3DBrainTissueSegmentation/test/"

In [17]:
MRI_list = os.listdir(HeadMRI_DIR)
Label_list = os.listdir(Headlabel_DIR)
print(MRI_list)
print(Label_list)

['NIFTI_1.nii', 'NIFTI_2.nii', 'NIFTI_3.nii', 'NIFTI_4.nii', 'NIFTI_5.nii', 'NIFTI_6.nii', 'NIFTI_7.nii']
['Label_1.nii', 'Label_2.nii', 'Label_3.nii', 'Label_4.nii', 'Label_5.nii', 'Label_6.nii', 'Label_7.nii']


In [67]:
img = None
msk = None
for i in range(0, len(MRI_list)):
    img=np.array(nib.load(HeadMRI_DIR + MRI_list[i]).get_fdata())
    msk=np.array(nib.load(Headlabel_DIR + Label_list[i]).get_fdata())
    img_scale=scaler.fit_transform(img.reshape(-1, img.shape[-1])).reshape(img.shape)
    print(img.max())
    #print(img_scale.shape)

1901.836669921875
2624.56982421875
2475.04833984375
2091.745849609375
2355.335205078125
3452.623779296875
3451.0


In [68]:
img=np.array(nib.load(HeadMRI_DIR + MRI_list[0]).get_fdata())
msk=np.array(nib.load(Headlabel_DIR + Label_list[0]).get_fdata())
img_scale=scaler.fit_transform(img.reshape(-1, img.shape[-1])).reshape(img.shape)

label = msk[0:240, 5:245, 5:185]
print(label.shape)
Y = label[0: 60, 0:60, 0:60]
print(Y.shape)
img_scale.dtype


(240, 240, 180)
(60, 60, 60)


dtype('float64')

In [69]:
reshape_img = img[0:240, 5:245, :]
print(reshape_img.shape)


(240, 240, 192)


## Get Equally Sliced Subvolume, Size(80,80,64)

In [17]:
orig_x = 240
orig_y = 240
orig_z = 192
step_x = 40
step_y = 40
step_z = 32
output_x = 80
output_y = 80
output_z = 64
num_classes = 6

for i in range(0, len(MRI_list)):
    filename = MRI_list[i]
    img=np.array(nib.load(HeadMRI_DIR + MRI_list[i]).get_fdata())
    msk=np.array(nib.load(Headlabel_DIR + Label_list[i]).get_fdata())
    img_scale=scaler.fit_transform(img.reshape(-1, img.shape[-1])).reshape(img.shape)
    
    #Reshape to 240x240x192
    image = img_scale[0:240, 5:245, :]
    label = msk[0:240, 5:245, :]

        # Initialize features and labels with `None`
    X = None
    Y = None

    for x in range(0, orig_x-output_x+1, step_x):
        for y in range(0, orig_y-output_y+1, step_y):
            for z in range(0, orig_z-output_z+1, step_z):

                start_x = x
                start_y = y
                start_z = z

                Y = label[start_x: start_x + output_x,start_y: start_y + output_y,start_z: start_z + output_z]
                    
                Y = keras.utils.to_categorical(Y, num_classes=num_classes)

                X = np.copy(image[start_x: start_x + output_x,start_y: start_y + output_y,start_z: start_z + output_z])
                    
                    # change dimension of X
                    # from (x_dim, y_dim, z_dim, num_channels)
                    # to (num_channels, x_dim, y_dim, z_dim)
                X = np.expand_dims(X, axis=0)
                    
                    # change dimension of y
                    # from (x_dim, y_dim, z_dim, num_classes)
                    # to (num_classes, x_dim, y_dim, z_dim)
                Y = np.moveaxis(Y, 3, 0)

                    #excludes the background class
                Y = Y[1:, :, :, :]

                file=filename \
                +"_x_"+str(start_x) \
                +"_y_"+str(start_y) \
                +"_z_"+str(start_z) \
                +"_"

                import h5py

                destination = os.path.join("D:/DataSet/HeadSeg/Raw_HeadSet", f"{file}.h5")

                if os.path.exists(destination):
                    # File already exists, do something
                    pass

                else:
                    os.makedirs(os.path.dirname(destination), exist_ok= True)
                    with h5py.File(destination, "w") as f1:
                        dset1 = f1.create_dataset("x", (1, output_x, output_y, output_z), dtype='float64', data=X)
                        dset2 = f1.create_dataset("y", (5, output_x, output_y, output_z), dtype='uint8', data=Y)
                        f1.close()
print("Done")

Done


## Get Random Subvolumn, Size(80,80,64) Must contain CSF, GM, WM

In [14]:
def get_random_sub_volume(image, label, filename = None,
                   orig_x = 240, orig_y = 240, orig_z = 192, 
                   output_x = 80, output_y = 80, output_z = 64,
                   num_classes = 6, max_tries = 100, brain_ratio = 0.8):

    # Initialize features and labels with `None`
    X = None
    Y = None

    tries = 0    

    while tries < max_tries:
        # randomly sample sub-volume by sampling the corner voxel
        # hint: make sure to leave enough room for the output dimensions!
        start_x = np.random.randint(0, orig_x - output_x+1)
        start_y = np.random.randint(0, orig_y - output_y+1)
        start_z = np.random.randint(0, orig_z - output_z+1)

        # extract relevant area of label
        Y = label[start_x: start_x + output_x,
                  start_y: start_y + output_y,
                  start_z: start_z + output_z]
        
        #print(Y.max())

        #check the ratio of volume contains brain volume a.k.a CSF, GM, WM
        brain = np.count_nonzero(np.array(Y) >= 3) / (80 * 80 * 64)
        #print("brain is " + f"{brain}")
        # increment tries counter
        tries += 1

        # if background ratio is below the desired threshold,
        # use that sub-volume.
        # otherwise continue the loop and try another random sub-volume
        if brain >= brain_ratio:

            # One-hot encode the categories.
            # This adds a 4th dimension, 'num_classes'
            # (output_x, output_y, output_z, num_classes)
            Y = keras.utils.to_categorical(Y, num_classes=num_classes)

            # make copy of the sub-volume
            X = np.copy(image[start_x: start_x + output_x,
                              start_y: start_y + output_y,
                              start_z: start_z + output_z])
            
            # change dimension of X
            # from (x_dim, y_dim, z_dim, num_channels)
            # to (num_channels, x_dim, y_dim, z_dim)
            #X = np.moveaxis(X, 3, 0)
            X = np.expand_dims(X, axis=0)

            # change dimension of y
            # from (x_dim, y_dim, z_dim, num_classes)
            # to (num_classes, x_dim, y_dim, z_dim)
            Y = np.moveaxis(Y, 3, 0)

            # take a subset of y that excludes the background class
            # in the 'num_classes' dimension
            Y = Y[1:, :, :, :]

            file=filename \
                +"_random" \
                +"_x_"+str(start_x) \
                +"_y_"+str(start_y) \
                +"_z_"+str(start_z)
            
            print(file + " brain ratio: " + str(brain))
            print(f"Image value maximum: {X.max()}, Image shape: {X.shape}")
            print(f"Label value maximum: {Y.max()}, label shape: {X.shape}")
            
            
            return X, Y, file

    # if we've tried max_tries number of samples
    # Give up in order to avoid looping forever.
    print(f"Tried {tries} times to find a sub-volume. Giving up...")

In [18]:
output_x = 80
output_y = 80
output_z = 64

for i in range(0, len(MRI_list)):
    file_name = MRI_list[i]
    img=np.array(nib.load(HeadMRI_DIR + MRI_list[i]).get_fdata())
    msk=np.array(nib.load(Headlabel_DIR + Label_list[i]).get_fdata())
    img_scale=scaler.fit_transform(img.reshape(-1, img.shape[-1])).reshape(img.shape)
    
    #Reshape to 240x240x192
    #img = img_scale[0:240, 5:245, :]
    #msk = msk[0:240, 5:245, :]

    count = 0

    while count < 25:
        
        X, Y , file = get_random_sub_volume(image = img_scale, label = msk, filename=file_name, brain_ratio=0.8)
        

        destination = os.path.join("D:/DataSet/HeadSeg/Raw_HeadSet", f"{file}.h5")

        if os.path.exists(destination):
            # File already exists, do something
            pass

        else:
            os.makedirs(os.path.dirname(destination), exist_ok= True)
            with h5py.File(destination, "w") as f1:
                dset1 = f1.create_dataset("x", (1, output_x, output_y, output_z), dtype='float64', data=X)
                dset2 = f1.create_dataset("y", (5, output_x, output_y, output_z), dtype='uint8', data=Y)
                f1.close()

        count += 1
    

NIFTI_1.nii_random_x_143_y_80_z_93 brain ratio: 0.78622802734375
NIFTI_1.nii_random_x_98_y_75_z_5 brain ratio: 0.547841796875
NIFTI_1.nii_random_x_132_y_77_z_65 brain ratio: 0.9760107421875
NIFTI_1.nii_random_x_95_y_18_z_84 brain ratio: 0.69599853515625
NIFTI_1.nii_random_x_48_y_88_z_46 brain ratio: 0.5538037109375
NIFTI_2.nii_random_x_49_y_21_z_44 brain ratio: 0.51330322265625
NIFTI_2.nii_random_x_140_y_34_z_41 brain ratio: 0.57931884765625
NIFTI_2.nii_random_x_79_y_30_z_58 brain ratio: 0.87241943359375
NIFTI_2.nii_random_x_83_y_78_z_66 brain ratio: 0.7949951171875
NIFTI_2.nii_random_x_94_y_101_z_80 brain ratio: 0.57045166015625
NIFTI_3.nii_random_x_123_y_58_z_114 brain ratio: 0.63586669921875
NIFTI_3.nii_random_x_127_y_53_z_52 brain ratio: 0.94825927734375
NIFTI_3.nii_random_x_71_y_45_z_115 brain ratio: 0.50937744140625
NIFTI_3.nii_random_x_122_y_91_z_12 brain ratio: 0.53525146484375
NIFTI_3.nii_random_x_75_y_41_z_17 brain ratio: 0.5126171875
NIFTI_4.nii_random_x_87_y_43_z_96 brain r

## Get random volumes from Augmentation Data

In [12]:
AUG_MRI = AUGMENT_DIR + "MRI/"
AUG_Label = AUGMENT_DIR + "Label/"

AUG_MRI_list = sort(os.listdir(AUG_MRI))
AUG_Label_list = sort(os.listdir(AUG_Label))


In [16]:
output_x = 80
output_y = 80
output_z = 64

for i in range(0, len(AUG_MRI_list)):
    file_name = AUG_MRI_list[i]
    img=np.load(AUG_MRI + AUG_MRI_list[i])
    msk=np.load(AUG_Label + AUG_Label_list[i])
    img_scale=scaler.fit_transform(img.reshape(-1, img.shape[-1])).reshape(img.shape)
    
    ##Reshape to 240x240x192
    #img = img_scale[0:240, 5:245, :]
    #msk = msk[0:240, 5:245, :]

    count = 0

    while count < 15:
        
        X, Y , file = get_random_sub_volume(image = img, label = msk, 
                                            orig_x=img.shape[0], orig_y=img.shape[1],orig_z=img.shape[2], 
                                            filename=file_name,brain_ratio= 0.75)

        destination = os.path.join("D:/DataSet/HeadSeg/Raw_HeadSet/Augmentation/", f"{file}.h5")

        if os.path.exists(destination):
            # File already exists, do something
            pass

        else:
            os.makedirs(os.path.dirname(destination), exist_ok= True)
            with h5py.File(destination, "w") as f1:
                dset1 = f1.create_dataset("x", (1, output_x, output_y, output_z), dtype='float64', data=X)
                dset2 = f1.create_dataset("y", (5, output_x, output_y, output_z), dtype='uint8', data=Y)
                f1.close()

        count += 1
    

NIFTI_1.nii_expand.npy_random_x_90_y_57_z_102 brain ratio: 0.91398681640625
NIFTI_1.nii_expand.npy_random_x_113_y_53_z_113 brain ratio: 0.7941162109375
NIFTI_1.nii_expand.npy_random_x_132_y_110_z_50 brain ratio: 0.845205078125
NIFTI_1.nii_expand.npy_random_x_84_y_72_z_37 brain ratio: 0.8170361328125
NIFTI_1.nii_expand.npy_random_x_126_y_101_z_66 brain ratio: 0.9192578125
NIFTI_1.nii_expand.npy_random_x_92_y_79_z_93 brain ratio: 0.927353515625
NIFTI_1.nii_expand.npy_random_x_112_y_47_z_82 brain ratio: 0.93760498046875
NIFTI_1.nii_expand.npy_random_x_142_y_121_z_67 brain ratio: 0.766923828125
NIFTI_1.nii_expand.npy_random_x_145_y_78_z_47 brain ratio: 0.90057861328125
NIFTI_1.nii_expand.npy_random_x_124_y_91_z_60 brain ratio: 0.95953369140625
NIFTI_1.nii_expand.npy_random_x_60_y_73_z_62 brain ratio: 0.83705078125
NIFTI_1.nii_expand.npy_random_x_118_y_93_z_82 brain ratio: 0.92324951171875
NIFTI_1.nii_expand.npy_random_x_148_y_57_z_66 brain ratio: 0.86868408203125
NIFTI_1.nii_expand.npy_ran

In [ ]:
img=np.array(nib.load(HeadMRI_DIR + MRI_list[0]).get_fdata())
msk=np.array(nib.load(Headlabel_DIR + Label_list[0]).get_fdata())
img_scale=scaler.fit_transform(img.reshape(-1, img.shape[-1])).reshape(img.shape)

label = msk[0:240, 5:245, 5:185]
print(label.shape)
Y = label[0: 60, 0:60, 0:60]
print(Y.shape)
img_scale.dtype